In [3]:
import pandas as pd
from pandas import option_context
import numpy as np
import requests
import seaborn as sns
import datetime
import cliffsDelta as cd

from tqdm import tqdm
from statsmodels.stats.inter_rater import fleiss_kappa,aggregate_raters
from scipy.stats import mannwhitneyu,shapiro,ttest_ind

In [4]:
QUERY_ROOT = "https://api.github.com"
TOKEN = '' # write your GitHub API key here
HEADERS={'Authorization':'token '+TOKEN}

## Section 3

### Question 1

#### Write an automated Python script to identify the GitHub Apps in the list of accounts, by calling the GitHub Users API and extracting the required information to make this decision. 

#### More information: You can search for the "type" key in the obtianed JSON result, If it is "Bot", then it is an app, if it is "User", then it is an account.

In [5]:
# 1. Read the given .csv file into a pandas DataFrame (df).
# 2. Make sure there are 2 columns, one column name is "contributor" that has the name of the contributor, 
#    and another column name is "bothunter" that you will use later on in this assignment.
# 3. Display the DataFrame.

# YOUR CODE BELOW
df = pd.read_csv('homebrew_homebrew-core_1/homebrew_homebrew-core_1.csv')
print(df)

                contributor bothunter
0            flyio-buildbot       Bot
1         blockstack-devops       Bot
2                SMillerDev     Human
3                    petere     Human
4                 bfontaine     Human
5                   zuisong     Human
6          asyncapi-bot-eve       Bot
7            EricFromCanada     Human
8                   Moulick     Human
9               BrewTestBot       Bot
10                 carlocab     Human
11                  ddollar     Human
12                  seratch     Human
13                   sferik     Human
14                  samford     Human
15            jesseduffield     Human
16                    claui     Human
17               jeremylong     Human
18                  tchajed     Human
19          FnControlOption     Human
20         flyio-flyctl-bot       Bot
21                     avsm     Human
22              denysdovhan     Human
23                 derrabus     Human
24  github-merge-queue[bot]       Bot
25          

In [6]:
# 1. Store the names of the contributors in a list using the code snippet: 'df.contributor.to_list()'
# 2. print the list

# YOUR CODE BELOW
contributors = df.contributor.to_list()
print(contributors)

['flyio-buildbot', 'blockstack-devops', 'SMillerDev', 'petere', 'bfontaine', 'zuisong', 'asyncapi-bot-eve', 'EricFromCanada', 'Moulick', 'BrewTestBot', 'carlocab', 'ddollar', 'seratch', 'sferik', 'samford', 'jesseduffield', 'claui', 'jeremylong', 'tchajed', 'FnControlOption', 'flyio-flyctl-bot', 'avsm', 'denysdovhan', 'derrabus', 'github-merge-queue[bot]', 'jidicula', 'bevanjkay', 'branchvincent', 'nevack', 'mkhl', 'wing328', 'stgraber', 'soloio-bot', 'matchai', 'kevinAlbs', 'curquiza', 'dandavison', 'pulumi-bot', 'epistrephein', 'github-actions[bot]']


In [7]:
# 1. Using a for loop, iterate over each contributor in the list and query the GitHub Users API.
#    You can use "query = f'{QUERY_ROOT}/users/{contributor}'", where QUERY_ROOT is defined at the beginning of this notebook 
#    above and 'contributor' is each individul contributor from the list
# 2. Get the response using 'response = requests.get(query, headers=HEADERS)'.
# 3. convert the response to JSON using 'json_response = response.json()'.
# 4. Iterate over this JSON response and get the value of the 'type' key. If it is "Bot" then the contributor is an App, 
#    if "User" then the contributor is an account. You should boolean values to indicate if the contributor is an App (True) or User/Organisation (False)
# 5. Save these results in list of dictionary of the form [{'contributor': <contributor name>, 'app': <boolean value>}, {...}, {...}, {...}].
#    Lets call this list as "app_list_dict"
# 6. Finally convert this list of dictionary to DataFrame by writing 'pd.DataFrame.from_dict(app_dict)'
# 7. Display the DataFrame. This should have two columns - contributor and app

# YOUR CODE BELOW
app_list_dict = []
for contributor in contributors:
    query = f'{QUERY_ROOT}/users/{contributor}'
    response = requests.get(query, headers=HEADERS)
    json_response = response.json()
    app_dict = {'contributor': contributor}
    if json_response['type'] == 'Bot':
        app_dict['app'] = True
    else:
        app_dict['app'] = False
    app_list_dict.append(app_dict)

df_dict = pd.DataFrame.from_dict(app_list_dict)
df_dict.to_csv('github_result.csv')
print(df_dict)

                contributor    app
0            flyio-buildbot  False
1         blockstack-devops  False
2                SMillerDev  False
3                    petere  False
4                 bfontaine  False
5                   zuisong  False
6          asyncapi-bot-eve  False
7            EricFromCanada  False
8                   Moulick  False
9               BrewTestBot  False
10                 carlocab  False
11                  ddollar  False
12                  seratch  False
13                   sferik  False
14                  samford  False
15            jesseduffield  False
16                    claui  False
17               jeremylong  False
18                  tchajed  False
19          FnControlOption  False
20         flyio-flyctl-bot  False
21                     avsm  False
22              denysdovhan  False
23                 derrabus  False
24  github-merge-queue[bot]   True
25                 jidicula  False
26                bevanjkay  False
27            branch

### Question 2

#### Add an app column in the CSV file to store this information. You can write True if it is an app and False if it is not.


In [8]:
# Merge the app DataFrame to df by writing 'pd.merge(df, <app df>, on='contributor'). This is similar to SQL join on primary key 'contributor'.
# The resultant df should have 3 columns - contributor, bothunter and app.

# YOUR CODE BELOW
result_df = pd.merge(df, df_dict, on='contributor')
print(result_df)

                contributor bothunter    app
0            flyio-buildbot       Bot  False
1         blockstack-devops       Bot  False
2                SMillerDev     Human  False
3                    petere     Human  False
4                 bfontaine     Human  False
5                   zuisong     Human  False
6          asyncapi-bot-eve       Bot  False
7            EricFromCanada     Human  False
8                   Moulick     Human  False
9               BrewTestBot       Bot  False
10                 carlocab     Human  False
11                  ddollar     Human  False
12                  seratch     Human  False
13                   sferik     Human  False
14                  samford     Human  False
15            jesseduffield     Human  False
16                    claui     Human  False
17               jeremylong     Human  False
18                  tchajed     Human  False
19          FnControlOption     Human  False
20         flyio-flyctl-bot       Bot  False
21        

### Question 3

#### Report on the total number of GitHub Apps, and User accounts present in the list of accounts of your dataset.

In [9]:
nbr_of_apps = result_df[result_df['app'] == True].shape[0]
nbr_of_users = result_df[result_df['app'] == False].shape[0]

print(f"Number of apps: {nbr_of_apps}")
print(f"Number of users: {nbr_of_users}")


Number of apps: 2
Number of users: 38


## Section 4

### Question 1

#### Write an automated script to report on the prediction of the GitHub accounts in your dataset, using BoDeGHa and RABBIT bot identification tools. For each tool, your script should add an extra column to the CSV ﬁle of your dataset, reporting the prediction (bot, human, unknown) of the considered tool on the considered account.

#### More information:
You can write an automated script to execute BoDeGHa and RABBIT, and save the results in a DataFrame. For BoDeGHa, you can ask the tool to give the output in CSV format, save it as a csv file and read it in a DataFrame. For RABBIT, you can use its internal argument to store the results in a .csv file and read it in a DataFrame. Remember to read their Documentation (Readme.md) for their input arguments and example outputs before using these tools.  
Similar to what you did in Section 2 - Question 1 and 2, add an additional column named 'bodegha' to store the predictions 
given by BoDeGHa and add another column called 'rabbit' to store the preditions given by RABBIT. 

In [10]:
# Read the .csv file that has the predicitons given by BoDeGHa.

# YOUR CODE BELOW
bodega_df = pd.read_csv('bodegha_predictions.csv')

unknowns = []

for user in df['contributor']:
    if user not in bodega_df['account'].tolist():
        unknowns.append(user)


#for every element in unknowns add a record in bodega_df with 'Unknown' as prediction
for unknown in unknowns:
    bodega_df.loc[len(bodega_df.index)] = [unknown, 'Unknown']

bodega_df = bodega_df.rename(columns={'account':'contributor', 'prediction':'bodegha'})
print(bodega_df)


                contributor  bodegha
0                  curquiza      Bot
1                jeremylong      Bot
2                pulumi-bot      Bot
3               BrewTestBot    Human
4            EricFromCanada    Human
5                   Moulick    Human
6                SMillerDev    Human
7                 bevanjkay    Human
8             branchvincent    Human
9                  carlocab    Human
10                 derrabus    Human
11                  samford    Human
12                  zuisong    Human
13      github-actions[bot]      Bot
14           flyio-buildbot  Unknown
15        blockstack-devops  Unknown
16                   petere  Unknown
17                bfontaine  Unknown
18         asyncapi-bot-eve  Unknown
19                  ddollar  Unknown
20                  seratch  Unknown
21                   sferik  Unknown
22            jesseduffield  Unknown
23                    claui  Unknown
24                  tchajed  Unknown
25          FnControlOption  Unknown
2

In [11]:
# merge it to df - Now your df should have the following four columns - contributor, bothunter, app, bodegha

# YOUR CODE BELOW
final_df = pd.merge(result_df, bodega_df, on='contributor')
print(final_df)




                contributor bothunter    app  bodegha
0            flyio-buildbot       Bot  False  Unknown
1         blockstack-devops       Bot  False  Unknown
2                SMillerDev     Human  False    Human
3                    petere     Human  False  Unknown
4                 bfontaine     Human  False  Unknown
5                   zuisong     Human  False    Human
6          asyncapi-bot-eve       Bot  False  Unknown
7            EricFromCanada     Human  False    Human
8                   Moulick     Human  False    Human
9               BrewTestBot       Bot  False    Human
10                 carlocab     Human  False    Human
11                  ddollar     Human  False  Unknown
12                  seratch     Human  False  Unknown
13                   sferik     Human  False  Unknown
14                  samford     Human  False    Human
15            jesseduffield     Human  False  Unknown
16                    claui     Human  False  Unknown
17               jeremylong 

In [12]:
# Read the predictions given by RABBIT from the corresponding .csv file 

# YOUR CODE BELOW
rabbit_df = pd.read_csv('homebrew_homebrew-core_1/rabbit_predictions.csv')
rabbit_df = rabbit_df.rename(columns={'account':'contributor', 'prediction':'rabbit'})
#drop index and confidence columns
rabbit_df = rabbit_df.drop(columns=['index', 'confidence'])
print(rabbit_df)

                contributor rabbit
0            flyio-buildbot    bot
1         blockstack-devops    bot
2                SMillerDev  human
3                    petere  human
4                 bfontaine  human
5                   zuisong  human
6          asyncapi-bot-eve    bot
7            EricFromCanada  human
8                   Moulick  human
9               BrewTestBot    bot
10                 carlocab  human
11                  ddollar  human
12                  seratch  human
13                   sferik  human
14                  samford  human
15            jesseduffield  human
16                    claui  human
17               jeremylong  human
18                  tchajed  human
19          FnControlOption  human
20         flyio-flyctl-bot    bot
21                     avsm  human
22              denysdovhan  human
23                 derrabus  human
24  github-merge-queue[bot]    app
25                 jidicula  human
26                bevanjkay  human
27            branch

In [13]:
# merge it to df - Now your df should have the following five columns - contributor, bothunter, app, bodegha, rabbit

# YOUR CODE BELOW
rabbit_full_df = pd.merge(final_df, rabbit_df, on='contributor')
print(rabbit_full_df)



                contributor bothunter    app  bodegha rabbit
0            flyio-buildbot       Bot  False  Unknown    bot
1         blockstack-devops       Bot  False  Unknown    bot
2                SMillerDev     Human  False    Human  human
3                    petere     Human  False  Unknown  human
4                 bfontaine     Human  False  Unknown  human
5                   zuisong     Human  False    Human  human
6          asyncapi-bot-eve       Bot  False  Unknown    bot
7            EricFromCanada     Human  False    Human  human
8                   Moulick     Human  False    Human  human
9               BrewTestBot       Bot  False    Human    bot
10                 carlocab     Human  False    Human  human
11                  ddollar     Human  False  Unknown  human
12                  seratch     Human  False  Unknown  human
13                   sferik     Human  False  Unknown  human
14                  samford     Human  False    Human  human
15            jesseduffi

In [14]:
# Replace the prediction result given by rabbit from 'bot' to 'Bot', 'app' to 'Bot', 'human' to 'Human', 'unknown' to 'Unknown' to maintain consistency
# YOUR CODE BELOW
for prediction in rabbit_full_df['rabbit']:
    if prediction == 'bot' or prediction == 'app':
        rabbit_full_df['rabbit'] = rabbit_full_df['rabbit'].replace(prediction, 'Bot')
    elif prediction == 'human':
        rabbit_full_df['rabbit'] = rabbit_full_df['rabbit'].replace(prediction, 'Human')
    elif prediction == 'unknown':
        rabbit_full_df['rabbit'] = rabbit_full_df['rabbit'].replace(prediction, 'Unknown')


print(rabbit_full_df)
rabbit_full_df.to_csv('rabbit_full_df.csv')


                contributor bothunter    app  bodegha rabbit
0            flyio-buildbot       Bot  False  Unknown    Bot
1         blockstack-devops       Bot  False  Unknown    Bot
2                SMillerDev     Human  False    Human  Human
3                    petere     Human  False  Unknown  Human
4                 bfontaine     Human  False  Unknown  Human
5                   zuisong     Human  False    Human  Human
6          asyncapi-bot-eve       Bot  False  Unknown    Bot
7            EricFromCanada     Human  False    Human  Human
8                   Moulick     Human  False    Human  Human
9               BrewTestBot       Bot  False    Human    Bot
10                 carlocab     Human  False    Human  Human
11                  ddollar     Human  False  Unknown  Human
12                  seratch     Human  False  Unknown  Human
13                   sferik     Human  False  Unknown  Human
14                  samford     Human  False    Human  Human
15            jesseduffi

### Question 2

#### Add an extra column to the CSV ﬁle called 'manual' that has the label determined by you. For each account, of your own opinion on its nature (bot, human or unknown only in case of doubt) on the basis of a manual veriﬁcation of the contributor’s activities in the repository through the GitHub UI and through the GitHub Events API https://api.github.com/users/username/events, and taking into account the prediction made by the bot identification tools.

#### More information:
For each contributor, look at their activities in GitHub UI, look at their latest events using GitHub Events API - https://api.github.com/users/<contributor\>/events, to make a decision on their type. 

In addition, you should also write a very small reason why do you think they are bot or human (e.g., same activity at regular interval so Bot, files committed and code modified looks like Human, comments look like Human, comment look like Bot, test report looks automated so Bot, and so on....). This might be useful to answer the next question as well.

In [16]:
# Write your manual decision here
# example : 
# manual_list_dict = [{'contributor': 'tommens', 'manual': 'human'}, {....}, {....}, {....}]
# pd.DataFrame.from_dict(manual_list_dict)

# YOUR CODE BELOW

In [ ]:
# merge it with df of predictions. Now your df should have the following six columns
# contributor, bothunter, app, bodegha, rabbit, manual

# YOUR CODE BELOW


### Question 3

#### Study and report the purpose of these identified bots and humans in the repository assigned to you. 

For example, the purpose can be based on but not limited to - 

the type of activities that they are performing (releasing a version on every Sunday, updating the documentation), type of comments they are posting (reviewing code, test summary report), when are they triggered (when a new PR is created, when someone in the project ask the bot to merge the code) and so on. \textbf{Note:} The purpose can be other than what is provided in examples above. 


In [ ]:
'''
github-merge-queue[bot] : The purpose of the bot is to automatically merge PR or queue them. 
    If everything in the PR is ok it will merge, if not it will queue the PR.

github-actions[bot] : The purpose of the bot is to deploy all kind of actions on user's repositorys to automate their workflow.

'''

### Question 4

#### Create a column named 'type' to the CSV ﬁle and write your ﬁnal decision on the type of contributor. The ﬁnal decision on their type is the prediciton that majority of the raters predicted (you can ignore the 'Unknown'), in the case of contradiction, you can finalise it as Human.
#### Case 1: If two raters has mentioned a contributor as Bot and others mentioned it as 'Unknown', then the type of the account is Bot.  
#### Case 2: If 3 raters give 'Unknown' to a contributor then the prediction given by the fourth rater is final.
#### Case 3: If 2 raters give the prediciton as 'Bot' and 2 raters give it as 'Human', then the final decision is 'Human'
#### In this step, GitHub Apps should be marked as App and not as a Bot.

In [22]:
# Make a final decision on the type based on the predictions given by bothunter, bodegha, bothunter and your manual labelling. 
# Now your df should contain seven columns - contributor, bothunter, app, bodegha, rabbit, manual and type

# YOUR CODE BELOW
results = []
for index, row in rabbit_full_df.iterrows():
    contributor = row['contributor']
    predictions = row[['bothunter', 'bodegha', 'rabbit']]
    app = row['app']
    human_occurence = 0
    unknown_occurences = 0
    bot_occurences = 0
    for prediction in predictions:
        if prediction == 'Human':
            human_occurence += 1
        elif prediction == 'Unknown':
            unknown_occurences += 1
        elif prediction == 'Bot':
            bot_occurences += 1

    if human_occurence < bot_occurences:
        if app:
            results.append({'contributor': contributor, 'type': 'App'})
        else:
            results.append({'contributor': contributor, 'type': 'Bot'})
    
    else:
        results.append({'contributor': contributor, 'type': 'Human'})

results_df = pd.DataFrame.from_dict(results)

final_decision = pd.merge(rabbit_full_df, results_df, on='contributor')

print(final_decision)

                contributor bothunter    app  bodegha rabbit   type
0            flyio-buildbot       Bot  False  Unknown    Bot    Bot
1         blockstack-devops       Bot  False  Unknown    Bot    Bot
2                SMillerDev     Human  False    Human  Human  Human
3                    petere     Human  False  Unknown  Human  Human
4                 bfontaine     Human  False  Unknown  Human  Human
5                   zuisong     Human  False    Human  Human  Human
6          asyncapi-bot-eve       Bot  False  Unknown    Bot    Bot
7            EricFromCanada     Human  False    Human  Human  Human
8                   Moulick     Human  False    Human  Human  Human
9               BrewTestBot       Bot  False    Human    Bot    Bot
10                 carlocab     Human  False    Human  Human  Human
11                  ddollar     Human  False  Unknown  Human  Human
12                  seratch     Human  False  Unknown  Human  Human
13                   sferik     Human  False  Un

### Question 5 (a)

#### Compute and report the Fleiss Kappa interrater agreement score between the labels computed by all bot identiﬁcation tools.

##### For example:

converted_df = aggregate_raters(np.array(df[['rater1','rater2']]))  
kappa = fleiss_kappa(converted_df[0])  
print('kappa')  

#### a. between bothunter and rabbit

In [28]:
converted_df1 = aggregate_raters(np.array(final_decision[['bothunter','rabbit']]))
kappa1 = fleiss_kappa(converted_df1[0])
print(f"Kappa: {kappa1}")

Kappa: 1.0


#### b. between bothunter and bodegha

In [27]:
converted_df2 = aggregate_raters(np.array(final_decision[['bothunter','bodegha']]))
kappa2 = fleiss_kappa(converted_df2[0])
print(f"Kappa: {kappa2}")

Kappa: -0.19772844605059373


#### c. between bodegha and rabbit

In [26]:
converted_df3 = aggregate_raters(np.array(final_decision[['bodegha','rabbit']]))
kappa3 = fleiss_kappa(converted_df3[0])
print(f"Kappa: {kappa3}")

Kappa: -0.19772844605059373


#### d. between bothunter, bodegha and rabbit

In [30]:
converted_df4 = aggregate_raters(np.array(final_decision[['bothunter', 'bodegha', 'rabbit']]))
kappa4 = fleiss_kappa(converted_df4[0])
print(f"Kappa: {kappa4}")

Kappa: 0.13604766633565038


### Question 5 (b)
#### Report on
#### a. What is you interpretation from the kappa values (use the table provided in the description document)? what do you understand?

In [ ]:
'''
For the first pair of raters the kappa is 1.0 it means that we have a perfect agreement between the two raters. Globally they have the same predictions
For the second pair of raters the kappa is -0.19 it means that we have no agreement between the two raters. Globally they have different predictions
For the third pair of raters the kappa is -0.19 it means that we have no agreement between the two raters. Globally they have different predictions
For the fourth pair of raters the kappa is 0.13 it means that they slightly agree. Globally they have different predictions
'''

#### b. Based on your previous analysis, which bot identification tool would you use, if you want to predict if a contributor is a human or bot? Motivate your answer

In [ ]:
'''
As soon as bodegha is used the kappa is below 0.2. It means that the predictions are not reliable.
When we use bothunter and rabbit the kappa is 1.0 It means that the predictions are reliable.
So we can say that bodegha is not reliable for the predictions. I would use bothunter and rabbit for the predictions.
'''

## Section 5

#### Query the GitHub Events API to get maximum possible events for each contributor (maximum is 300 events).  
#### You can use the following query: '{QUERY_ROOT}/users/{contributor}/events?per_page=100&page={page}' where page = 1 or 2 or 3  
#### As you did in earlier task, make a request to GitHub Events API with the required headers, convert the response to JSON, extract the following fields and store in a DataFrame:
Event id - json_response['event_id']  
Name of the contributor - json_response['actor']['login']  
Name of the repository - json_response['repository']['name']  
Type of event - json_response['type']  
Time of event - json_response['created_at']

#### Hint: Query the GitHubEvents API for all contributors. Append the obtained JSON result to a list of JSONs. Process one dictionary after other to get your required fields and store them in another list of dictionaries. Finally convert this dictionaty to DataFrame. OR You can directly convert the list of JSONs to DataFrame using pd.json_normalize() and select the required columns alone.

#### The resultant dataframe should have all the recent events performed by all the contributors and the following columns should be present in them - event_id, contributor, repo_name, event_type, created_at.


In [36]:
# YOUR CODE BELOW
events = []
for contributor in final_decision['contributor']:
    for k in range(1, 4):
        query = f'{QUERY_ROOT}/users/{contributor}/events?per_page=100&page={k}'
        response = requests.get(query, headers=HEADERS)
        event = response.json()
        for event in json_response:
            event_dict = {}
            event_dict["event_id"] = event["id"]
            event_dict["contributor"] = event["actor"]["login"]
            event_dict["repo_name"] = event["repo"]["name"]
            event_dict["event_type"] = event["type"]
            event_dict["created_at"] = event["created_at"]

            events.append(event_dict)

events_df = pd.DataFrame.from_dict(events)
print(events_df)

          event_id     contributor        repo_name   event_type  \
0      36777977652  flyio-buildbot  superfly/flyctl  CreateEvent   
1      36776807787  flyio-buildbot  superfly/flyctl  CreateEvent   
2      36776682568  flyio-buildbot  superfly/flyctl  CreateEvent   
3      36776610291  flyio-buildbot  superfly/flyctl  CreateEvent   
4      36772202633  flyio-buildbot  superfly/flyctl  CreateEvent   
...            ...             ...              ...          ...   
11995  35928144277  flyio-buildbot  superfly/flyctl  CreateEvent   
11996  35923678658  flyio-buildbot  superfly/flyctl  CreateEvent   
11997  35886219842  flyio-buildbot  superfly/flyctl  CreateEvent   
11998  35850879967  flyio-buildbot  superfly/flyctl  CreateEvent   
11999  35849615388  flyio-buildbot  superfly/flyctl  CreateEvent   

                 created_at  
0      2024-03-21T21:55:02Z  
1      2024-03-21T21:03:32Z  
2      2024-03-21T20:58:50Z  
3      2024-03-21T20:56:12Z  
4      2024-03-21T18:20:52Z  
...

### Question 1
#### Group each event into the following four categories

Issues: IssueCommentEvent, IssuesEvent  
Pull Requests: PullRequestEvent, PullRequestReviewCommentEvent  
Commits: CommitCommentEvent, PushEvent  
Repository: CreateEvent, DeleteEvent, ForkEvent, GollumEvent, MemberEvent, PublicEvent, ReleaseEvent, SponsorshipEvent, WatchEvent  

#### Hint:
1. Add a column called event_group that mentions which event group does that event belong to. Each event (row) should correspond to an event group.  
2. Then perform groupby on ['contributor','event_group'],  
3. use .agg (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) to count the number of events performed by each contributor in each group,  
4. use pivot with the required arguments (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). An example is given [here](#pivot_example) in question 2(b), and  
5. finally merge it with your 'type' field from the DataFrame you got in Section4

The final DataFrame should be looking like the following  
![](event_group.png)

In [ ]:
# In the resultant DataFrame - each row should corespond to a contributor, and the columns should have all the event groups  
# and the type of the contributor (that you decided in the previous DataFrame) and the values should be the number 
# of events of that event event group the contriubutor has performed.

issue_group = ['IssueCommentEvent', 'IssuesEvent']
pr_group = ['PullRequestEvent', 'PullRequestReviewCommentEvent']
commit_group = ['CommitCommentEvent', 'PushEvent']
repo_group = ['CreateEvent', 'DeleteEvent', 'ForkEvent', 'GollumEvent', 'MemberEvent', 'PublicEvent', 'ReleaseEvent', 'SponsorshipEvent', 'WatchEvent']

In [ ]:
# YOUR CODE BELOW


In [ ]:
# YOUR CODE BELOW


### Question 2 (a)

#### Compute the median number of events per event group for Bot+Apps and Humans and write in DataFrame.

Row should correspond to type (Bot_App and Human), Column should have Event group name and the values should be the median value of Bot_App or Human for that particular event group. An example is given below

In [ ]:
# For example:
medians = [{'event_group': 'event_group1', 'median': 'val1', 'type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [ ]:
# YOUR CODE BELOW

### Question 2 (b)

Plot a heatmap of the DataFrame using seaborn - 
1. First convert the dataframe to the required format using pivot, example is given below
2. plot using seaborn - sns.heatmap(df_medians, annot=True, vmin=0, vmax=300, cmap="crest"). More details: https://seaborn.pydata.org/generated/seaborn.heatmap.html)

#### pd.pivot example:
<a id='pivot_example'></a>

In [ ]:
# Main DataFrame
medians = [{'event_group': 'event_group1', 'median': 'val1', 'type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [ ]:
# pivot the main DataFrame
df_medians.pivot(index='type', columns='event_group', values='median')#.reset_index().rename_axis(None,axis=1)

event_group,event_group1,event_group2,event_group3,event_group4,event_group5
type,,,,,
Bot_app,val1,val3,val5,val7,NaN
Human,val2,val4,val6,NaN,val8


In [ ]:
# YOUR CODE BELOW

In [ ]:
# YOUR CODE BELOW


#### What is the difference that you observe between Bots+Apps and Humans?

#### What is the differnce that you observe between Event groups?

#### What is the difference that you observe between Bots+Apps and Humans and Event groups all considering at the same time?

### Question 3

#### Create box plots or boxen plots or violin plots or a combination of these to visualise the distribution of number of events in each event group. 
#### For more information you can visit - https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### For violin plots - https://seaborn.pydata.org/generated/seaborn.violinplot.html  
#### You should highlight the data points that correspond to bots using a stripplot in seaborn. https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### Interpret the results of the visualisation.

In [ ]:
# YOUR CODE BELOW - Visualize number of events from pull request event group


In [ ]:
# YOUR CODE BELOW - Visualise number of events from issue event group


In [ ]:
# YOUR CODE BELOW - Visualise number of events from commit event group


In [ ]:
# YOUR CODE BELOW - Visualise number of events from repo event group


### Question 4.1

#### Statistical identify whether number of events in each event group is normally distributed or not.
#### Null hypothesis - $H_0$: Sample comes from the data that has normal distribution.
#### Use Shapiro-Wilk test for this purpose. Use the p-value with a threshold of 0.05 to determine whether $H_0$ can be rejected with statistical significance or not. 

#### Use shapiro from scipy.stats to perform this test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

In [ ]:
# YOUR CODE BELOW

### Question 4.2
#### Determine if there is any statistical diﬀerence in number of events between the identified event groups. Perform this test on all pairs of event groups. 
#### Null hypothesis - $H_0$: Any two event group come from the same population.
#### If $H_0$ is rejected in Shapiro-Wilk test (at least for one of the two event groups considered for test), use the Mann-Whitney U statistical test for this purpose. If $H_0$ is not rejected in Shapiro-Wilk test, use the independent t-test for this purpose.  
#### In any case, use the p-value with a threshold of 0.01 to determine whether $H_0$ can be rejected with statistical significance. 

#### Use mannwhitneyu from scipy.stats to perform Mann-Whitney U test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) 
#### or 
#### ttest_ind from scipy.stats to perform independent t-test https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html.

#### More information:

You can pass the following arguments to mannwhitneyu from scipy.stats - (method='exact', nan_policy='omit'). For ttest_ind you can use - (nan_policy='omit')

In [ ]:
# YOUR CODE BELOW

### Question 5
#### Each time you reject the null hypothesis $H_0$, quantify the effect size of the diﬀerence between the groups using cliﬀ’s delta ($\delta$). 
#### To calculate cliﬀ’s delta, you can pass the list of values to cliﬀ delta.py file given in the repository. E.g., cliffsDelta.cliffsDelta(list of values). This will return the effect size and its interpretation (negligible, small, medium, large).

In [ ]:
# YOU CODE BELOW

### Qusetion 6

#### Create violinplots to visualise the distributions for the following metrics and answer the questions that follow it. You should highlight the data points corresponding to bots using a stripplot in seaborn.

#### Hint:

1. Merge you DataFrame of event with the type of contributor that you identified in section 3.
2. Write a function that you will use for all the seaborn plots. This way you can reduce duplication and update it easily.

In [ ]:
# YOUR CODE BELOW - function for seaborn plot


#### Total number of events

In [ ]:
# YOUR CODE BELOW

#### Number of event types

In [ ]:
# YOUR CODE BELOW

#### Mean number of events per event type

In [ ]:
# YOUR CODE BELOW

#### Mean number of events per event group

In [ ]:
# YOUR CODE BELOW

#### Gini coeﬃcient of time between consecutive event

In [ ]:
def time_diffs(time_stamps):
    diff_time_stamps = np.array([])
    for i in range(len(time_stamps)-1):
        diff = (time_stamps.iloc[i+1] - time_stamps.iloc[i])/pd.to_timedelta('1 second')
        diff_time_stamps = np.append(diff_time_stamps, [diff])
    return(diff_time_stamps)

In [ ]:
def gini(time_stamps):
    '''
    args: time_stamps - Series of time stamps for a contributor
    
    return: g_coeff - float that gives gini coefficient
    
    method: calculate the Gini coefficient of a numpy array
    '''
    array = time_diffs(time_stamps)
    array = array[array!=0]
    if len(array) == 0:
        return np.float64(0.0)
    array = array.flatten()
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    g_coeff = (np.sum((2 * index - n - 1) * array)) / (n * np.sum(array))
    
    return (g_coeff.round(4))

You can pass a Series of time-stamps to gini(time_stamps) function to get the Gini coeﬃcient
#### Hint:
1. Groupby the contributors on 'contributor' and use gini as .agg function on 'created_at'.
2. Merge the resultant DataFram with their type (the df that you got in section 4)
3. Make box/boxen/violin plots on those gini values

In [ ]:
# YOUR CODE BELOW


In [ ]:
# YOUR CODE BELOW


### Question 6.1

#### Do you ﬁnd any set of accounts having a diverging behaviour? (Hint: look for outliers.)

### Question 6.2

#### What is the type of the accounts that have diverging behaviour?

### Question 6.3

#### Do bots have different behaviour than apps? Motivate your answer

### Question 6.4

#### Do bots and apps have a diﬀerent behaviour from human accounts? (Hint: in the box plots ﬁnd where the bots and apps are located and find where the humans are located)

### Question 6.5

#### Do all bots (other than outliers) have the same behaviour? List the bots that have a diﬀerent behaviour. Mention your interpretation on the cause of this diverging behaviour compared to other bots?